In [2]:
"""
Perform all-sky scans with no spatial prior information.
Record the TS for each pixel.

This is a jupyter test.
The actual work is done with `01_get_versatile_background_allsky_scan.py`
"""

'\nPerform all-sky scans with no spatial prior information.\nRecord the TS for each pixel.\n\nThis is a jupyter test.\nThe actual work is done with `01_get_versatile_background_allsky_scan.py`\n'

In [3]:
import os
import sys
print("Python version: ", end=' ')
print(sys.version)

import numpy as np
import healpy as hp
# import histlite as hl
import csky as cy
import pandas as pd
from scipy import sparse

# import matplotlib.pyplot as plt
# from matplotlib import cm
# import matplotlib.colors as colors
############# comment out below two lines on clusters ##########
# %matplotlib inline
# %matplotlib notebook
################################################################
from glob import glob
timer = cy.timing.Timer()
time = timer.time

###### Local Import ######
import SETTING
paths = SETTING.PATH(osg=False)
print(paths)
LOCATION = paths.LOCATION
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR

from utils import *

Python version:  3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0]

#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [4]:
tw_batchSize_map = {
                 10:1000,
                 25:1000,
                 50:1000,
                 100:1000,
                 250:1000,
                 500:1000
                }

In [5]:
import argparse

######################### CONFIGURE ARGUEMENTS #############################
# p = argparse.ArgumentParser(description="Background Trials",
#                             formatter_class=argparse.RawTextHelpFormatter)
# p.add_argument("--grb_name", default="GRB180423A", type=str, help="Name of one GRB")
# p.add_argument("--batchNtrials", default=10, type=int, help="Number of trials in this batch")
# p.add_argument("--batchIndex", default=0, type=int, help="Index of current batch")
# p.add_argument("--tw_in_second", default=10, type=int, help="Length of the time window in seconds")
# p.add_argument("--ncpu", default=1, type=int, help="Number of CPU to give Csky")
# p.add_argument("--mode", default="production", type=str, help="Mode: production or testing")
# p.add_argument("--outfilename", default="", type=str, help="Output filename should have type .npz. Highly recommended on OSG")
# args = p.parse_args()
###########################################################################


### testing on jupyter ###

class args:
    grb_name = "GRB180423A"    # real healpix example
    # grb_name = "GRB190415A"    # fake healpix example 
    batchNtrials = 20
    batchIndex = 0
    tw_in_second = 10
    ncpu = 4
    mode = "testing"
    outfilename = ""

##########################

In [6]:
if tw_batchSize_map[args.tw_in_second] != args.batchNtrials and args.mode=="prodection":
    raise Exception("You are not using the recommended batchNtrials wrt this tw_in_second!")

In [7]:
print("\n===== Loading noHeaGRB list =====\n") 
if args.mode != "production" and LOCATION != "IC-OSG":
    ANA_DIR = cy.utils.ensure_dir(ANA_DIR + "/test")
# All times in days, all angles in degrees
try:
    print("Loading pkl in DATA_DID")
    # df = pd.read_pickle(DATA_DIR+"/grbwebgbm/grbweb_gbm_noHeaplix.pkl")
    df = pd.read_pickle(DATA_DIR+"/grbwebgbm/grbweb_gbm_noHeaplix_2297.pkl")
    
except:
    try:
        print("Loading pkl in current path")
        # df = pd.read_pickle("grbweb_gbm_noHeaplix.pkl")
        df = pd.read_pickle("grbweb_gbm_noHeaplix_2297.pkl")
        
    except:
        raise Exception("Cannot pd.reade_picle() the grbweb_gbm_noHeaplix.pkl.\n")

print("\n...Done\n")
    
grb_row = df.loc[df['grb_name'] == args.grb_name]
tw = args.tw_in_second/86400.
tw_start = grb_row.t_center - 0.5*tw
ra = grb_row.ra
dec = grb_row.dec
print("\n...Done\n")


===== Loading noHeaGRB list =====

Loading pkl in DATA_DID

...Done


...Done



In [13]:
print("\n===== Setting up csky =====\n")
data_dir = ICDATA_DIR
# data_filenames = sorted(glob(data_dir + '/IC86_20*.data_with_angErr.npy'))
data_filenames = sorted(glob(data_dir + '/IC86_20*.data.npy'))

# sig_filenames = sorted(glob(data_dir + '/IC86_2012.nu*_merged_with_angErr.npy'))
# load nue only to save memory, never used in this .py
sig_filenames = sorted(glob(data_dir + '/IC86_2012.nu*_merged.npy'))
grl_filenames = sorted(glob(data_dir + '/GRL/IC86_20*.data.npy'))

################ energy lower bound #############
min_log_e = np.log10(10)
#################################################
bins_sindec = np.linspace(-1, 1, 25+1)  
# bins_logenergy = np.linspace(min_log_e, 5, 30+1)
bins_logenergy = np.linspace(min_log_e, 4, 25+1)

data = [np.load(data_filename) for data_filename in data_filenames]
data = np.hstack(data)
sig = [np.load(sig_filename) for sig_filename in sig_filenames]
sig = np.hstack(sig)
grl = [np.load(grl_filename) for grl_filename in grl_filenames]
grl = np.hstack(grl)
if min_log_e is not None:
    data_mask = data['logE'] > min_log_e
    data = data[data_mask]
    sig_mask = sig['logE'] > min_log_e
    sig = sig[sig_mask]
    
dataset_spec = cy.selections.CustomDataSpecs.CustomDataSpec(data, sig, np.sum(grl['livetime']),
                                                     sindec_bins=bins_sindec,
                                                     logenergy_bins=bins_logenergy,
                                                     grl=grl, key='greco_v2.10', cascades=True)

ANA_DIR = cy.utils.ensure_dir(ANA_DIR)
# on OSG
# ana_dir = "./"
ana = cy.get_analysis(cy.selections.repo
                      , dataset_spec
                      , dir=ANA_DIR
                      , load_sig=False)  # to save memory  




===== Setting up csky =====

Setting up Analysis for:
greco_v2.10
Setting up greco_v2.10...
Energy PDF Ratio Model...
  * gamma = 4.0000 ...
Signal Acceptance Model...
  * gamma = 4.0000 ...
Done.


In [14]:
conf = {
    'ana': ana,
    #### llh basics: csky.conf
    'space': 'ps', # ps/fitps/template/prior
    'time': 'transient', # utf/lc/transient
    'energy': 'customflux', # fit/customflux
    'flux': cy.hyp.PowerLawFlux(2),
    #### inj.py - prior has some duplications against space's prior
    'sig': 'transient', # ps/tw/lc/transient/template/prior
    'full_sky': True,
    'extended': True,
    'mp_cpus': args.ncpu,
    'cut_n_sigma': 3
    }

cy.CONF.update(conf)

In [15]:
print("\n===== Generating seeds for current batch =====\n")   
src = cy.utils.Sources(
    ra=ra,
    dec=dec,
    deg=True,
    mjd=tw_start, 
    sigma_t=np.zeros_like(tw), 
    t_100=tw,  # in days
    # prior=[hl.heal.HealHist(healpix)],
    name=args.grb_name
)
sstr = cy.get_sky_scan_trial_runner(conf=cy.CONF
                                    ,nside=64
                                    ,src_tr=src)

rng=np.random.default_rng(abs(java_hash(src.name[0])))
seeds = rng.integers(int(1e9), size=int(2e8))[args.batchNtrials*args.batchIndex: args.batchNtrials*(args.batchIndex + 1)]
print("\n...Done\n")


===== Generating seeds for current batch =====


...Done



In [17]:
print("\n===== Scanning =====\n") 
all_pixel_TS = sparse.lil_matrix((len(seeds), hp.nside2npix(64)), dtype=float)
# for a short test on Mar 12, 2021
arr = []
with time("allsky scramble scan"):
    for no_trial, seed in enumerate(seeds):
        if no_trial % (len(seeds) // 10) == 0:
            print("Working on no_trial: {} \n".format(no_trial))
        # scan (3,49152): -log10p, TS, ns
        scan = sstr.get_one_scan(n_sig=0
                                 , poisson=False
                                 , seed=seed
                                 , TRUTH=False
                                 , mp_cpus=args.ncpu
                                 , logging=False)
        all_pixel_TS[no_trial] = scan[1]
        # for a short test on Mar 12, 2021
        arr.append(scan)
print("\n...Done\n")


===== Scanning =====

Working on no_trial: 0 

Working on no_trial: 2 

Working on no_trial: 4 

Working on no_trial: 6 

Working on no_trial: 8 

Working on no_trial: 10 

Working on no_trial: 12 

Working on no_trial: 14 

Working on no_trial: 16 

Working on no_trial: 18 


0:20:55.440708 elapsed.

...Done



In [18]:
brr = np.array(arr)

In [19]:
brr.shape

(20, 3, 49152)

In [12]:
assert 0:
    print("Stop here")

SyntaxError: invalid syntax (<ipython-input-12-1834b159b3cb>, line 1)

In [ ]:
print("\n===== Converting to scipy.sparse and Save to disk =====\n")
with time("To scipy.sparse npz"):
    hp_sparse = all_pixel_TS.tocsr()
    outfilename = args.outfilename
    if not outfilename:
        outfilename = "{}_batchSize{}_batchIndex{}_tw{}.npz".format(args.grb_name, 
                                                                    args.batchNtrials, 
                                                                    args.batchIndex, 
                                                                    args.tw_in_second)
## on locations other than OSG
    output_folder = cy.utils.ensure_dir(ANA_DIR+"/allsky_scan/no_prior_versatile/tw{}".format(args.tw_in_second))
    sparse.save_npz(output_folder+"/{}".format(outfilename)
                    ,hp_sparse)
## on OSG
#     sparse.save_npz("{}".format(outfilename)
#                     ,hp_sparse)           
##
print("######## All Done. ###########")